In [20]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [21]:
df = pd.read_csv('./tesla.com.csv')
df

,Unnamed: 0,sell_type,car_type,year,traction,price,km,range,max_vel,zero_hundred
0,0,Nuevo,Model S,2024,"Model S, tracción a las cuatro ruedas",89.100 €,Cuentakilómetros con 6196 km,634,250,"3,2"
1,1,Nuevo,Model S,2024,"Model S, tracción a las cuatro ruedas",99.600 €,Cuentakilómetros con 503 km,634,250,"3,2"
2,2,Nuevo,Model S,2024,Plaid,101.700 €,Cuentakilómetros con 3652 km,600,322,"2,1"
3,3,Nuevo,Model S,2024,"Model S, tracción a las cuatro ruedas",102.270 €,Cuentakilómetros con 99 km,634,250,"3,2"
4,4,Nuevo,Model S,2024,"Model S, tracción a las cuatro ruedas",102.270 €,0,634,250,"3,2"
...,...,...,...,...,...,...,...,...,...,...
181,181,Ocasion,Model X,2023,Plaid,85.500 €,Cuentakilómetros con 27.271 km,543,262,"2,6"
182,182,Ocasion,Model X,2023,Plaid,86.500 €,Cuentakilómetros con 21.471 km,543,262,"2,6"
183,183,Ocasion,Model X,2023,Plaid,87.600 €,Cuentakilómetros con 20.074 km,543,262,"2,6"
184,184,Ocasion,Model Y,2024,Model Y Tracción trasera,40.300 €,Cuentakilómetros con 734 km,455,217,"6,9"


In [22]:
df.isna().sum()

Unnamed: 0      0
sell_type       0
car_type        0
year            0
traction        0
price           0
km              0
range           0
max_vel         0
zero_hundred    0
dtype: int64

In [23]:
df.dtypes

Unnamed: 0       int64
sell_type       object
car_type        object
year             int64
traction        object
price           object
km              object
range            int64
max_vel          int64
zero_hundred    object
dtype: object

In [24]:
for column in df.columns:
    print(df[column].unique())
    print('=' * 50)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185]
['Nuevo' 'Ocasion']
['Model S' 'Model 3' 'Model X' 'Model Y']
[2024 2018 2020 2023 2021 2019 2022]
['Model S, tracción a las cuatro ruedas' 'Plaid'
 'Model 3 Tracción trasera' 'Model 3 Gran autonomía con tracción trasera'
 'Model X, tracción a 

In [25]:
df.drop(columns=["Unnamed: 0", "traction"], inplace=True)
df.drop_duplicates(inplace=True)
df['km'] = df['km'].str.extract(r'(\d+)').astype(float)
df['price'] = df['price'].str.replace('€', '', regex=False)
df['price'] = df['price'].astype(str).str.replace('.', '', regex=False).astype(float)  
df['zero_hundred'] = df['zero_hundred'].str.replace(',', '.', regex=True).astype(float)

df = pd.get_dummies(data=df, columns=['sell_type', 'car_type'], drop_first=True)

In [26]:
df.dtypes

year                   int64
price                float64
km                   float64
range                  int64
max_vel                int64
zero_hundred         float64
sell_type_Ocasion       bool
car_type_Model S        bool
car_type_Model X        bool
car_type_Model Y        bool
dtype: object

In [27]:
df

,year,price,km,range,max_vel,zero_hundred,sell_type_Ocasion,car_type_Model S,car_type_Model X,car_type_Model Y
0,2024,89100.0,6196.0,634,250,3.2,False,True,False,False
1,2024,99600.0,503.0,634,250,3.2,False,True,False,False
2,2024,101700.0,3652.0,600,322,2.1,False,True,False,False
3,2024,102270.0,99.0,634,250,3.2,False,True,False,False
4,2024,102270.0,0.0,634,250,3.2,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...
181,2023,85500.0,27.0,543,262,2.6,True,False,True,False
182,2023,86500.0,21.0,543,262,2.6,True,False,True,False
183,2023,87600.0,20.0,543,262,2.6,True,False,True,False
184,2024,40300.0,734.0,455,217,6.9,True,False,False,True


In [28]:
df.describe()

,year,price,km,range,max_vel,zero_hundred
count,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000
mean,2023.359756,58589.512195,1502.414634,533.225610,227.365854,5.102439
std,1.481531,29548.757291,2494.354829,58.885073,26.863213,1.430086
min,2018.000000,27000.000000,0.000000,430.000000,201.000000,2.100000
25%,2024.000000,38007.500000,0.000000,513.000000,201.000000,3.900000
50%,2024.000000,45705.000000,41.500000,513.500000,217.000000,5.900000
75%,2024.000000,83500.000000,2094.500000,576.000000,250.000000,6.100000
max,2024.000000,126870.000000,9716.000000,702.000000,322.000000,6.900000


In [29]:
X = df.drop(columns=['price'])  # Elimina la columna objetivo del conjunto de características
y = df['price']  # Define la variable objetivo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

TESTS

In [30]:
criterios = ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']
profundidades = [2, 3, 5, 10, 20, 100]

mejor_clf = {'profundidad':'',
              'criterio':'',
              'mae':'',
              'mse':'',
              'rmse':'',
              'r2':''}
mejor_rmse = 999999999999999

In [31]:
for criterio in criterios:
    for profundidad in profundidades:
        # Crear el modelo de RandomForest
        dtr = DecisionTreeRegressor(criterion=criterio, max_depth=profundidad, random_state=1337)
        
        # Entrenar el modelo
        dtr.fit(X_train, y_train)
        
        # Realizar las predicciones
        predicciones = dtr.predict(X_test)
        
        # Calcular las métricas
        mae = mean_absolute_error(y_test, predicciones)
        mse = mean_squared_error(y_test, predicciones)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, predicciones)
        
        if rmse <= mejor_rmse:
            mejor_rmse = rmse
            mejor_clf['profundidad'] = profundidad
            mejor_clf['criterio'] = criterio
            mejor_clf['mae'] = mae
            mejor_clf['mse'] = mse
            mejor_clf['rmse'] = rmse
            mejor_clf['r2'] = r2

In [32]:
mejor_clf

{'profundidad': 5,
 'criterio': 'poisson',
 'mae': np.float64(2242.873141351402),
 'mse': np.float64(9869658.207823247),
 'rmse': np.float64(3141.6012171857915),
 'r2': 0.9902606306536719}

In [33]:
dtr = DecisionTreeRegressor(criterion=mejor_clf['criterio'], max_depth=mejor_clf['profundidad'], random_state=1337)
dtr

DecisionTreeRegressor(criterion='poisson', max_depth=5, random_state=1337)

In [34]:
dtr.fit(X_train, y_train)
predicciones = dtr.predict(X_test)

In [35]:
X_test['predicted_price'] = predicciones
X_test['real_price'] = df.loc[X_test.index, 'price']
X_test

,year,km,range,max_vel,zero_hundred,sell_type_Ocasion,car_type_Model S,car_type_Model X,car_type_Model Y,predicted_price,real_price
137,2018,117.0,490,225,4.4,True,True,False,False,28700.000000,29100.0
57,2024,0.0,576,250,3.9,False,False,True,False,116183.333333,104370.0
25,2024,0.0,513,201,6.1,False,False,False,False,39517.826087,37520.0
13,2024,10.0,513,201,6.1,False,False,False,False,39517.826087,36380.0
20,2024,6818.0,513,201,6.1,False,False,False,False,36598.666667,36980.0
146,2023,6301.0,634,250,3.2,True,True,False,False,85825.000000,84400.0
66,2024,0.0,576,250,3.9,False,False,True,False,116183.333333,113770.0
54,2024,0.0,702,201,5.2,False,False,False,False,39517.826087,42770.0
78,2024,0.0,543,262,2.6,False,False,True,False,124370.000000,120970.0
91,2024,372.0,455,217,6.9,False,False,False,True,46446.000000,44590.0


In [36]:
# FIN DecisionTreeRegressor - Marcos